In [1]:
import pandas as pd
path = 'dbs/DataPreprocess.csv'
df1 = pd.read_csv(path)
df1

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [2]:
# 가정 ax + bx1 + cx2 >= y
# 나라이름 못곱한다. 곱하기가 안된다.
# 나이, 샐러리 곱해는지나 1000배 이상 차이난다.

# 평균 : 산술, 최빈, 중간
# 산술평균으로 대치

from sklearn.preprocessing import Imputer
import numpy as np

x = df1.values[: , : -1]
y = df1.values[:, -1]

imputer = Imputer(missing_values='NaN', strategy="mean", axis=0) # NaN 데이터를 세로축 산술평균 
imputer.fit(x[:,1:3]) # 문자열 평균 못하니 제외 fit는 훈련만 실제적용은 transform
imputer.transform(x[:,1:3])

# e+01 10의 1승 e+04 10의 4승
# -80 0에 가까운수

/Users/newdecide/.pyenv/versions/3.7.2/envs/3vm/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[4.40000000e+01, 7.20000000e+04],
       [2.70000000e+01, 4.80000000e+04],
       [3.00000000e+01, 5.40000000e+04],
       [3.80000000e+01, 6.10000000e+04],
       [4.00000000e+01, 6.37777778e+04],
       [3.50000000e+01, 5.80000000e+04],
       [3.87777778e+01, 5.20000000e+04],
       [4.80000000e+01, 7.90000000e+04],
       [5.00000000e+01, 8.30000000e+04],
       [3.70000000e+01, 6.70000000e+04]])

In [3]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=3, shuffle=True, random_state=0)

for train_index, test_index in cv.split(x): # 인덱스 확인
    print('test_index', test_index)
    print('train_index', train_index)

test_index [2 4 8 9]
train_index [0 1 3 5 6 7]
test_index [1 6 7]
train_index [0 2 3 4 5 8 9]
test_index [0 3 5]
train_index [1 2 4 6 7 8 9]


In [4]:
# 0x1 + bx2 + c = y
# ax1+bx2 = y 항이 더적어 더적은데이터로 구할 수 있다.
from sklearn.preprocessing import StandardScaler

sc_x = StandardScaler()
sc_x.fit(x[:,1:3])
sc_x.transform(x[:,1:3])

array([[ 0.71993143,  0.71101276],
       [-1.62367514, -1.36437583],
       [-1.21009751, -0.84552869],
       [-0.10722383, -0.24020701],
       [ 0.16849459,         nan],
       [-0.52080146, -0.49963059],
       [        nan, -1.01847774],
       [ 1.27136827,  1.31633443],
       [ 1.54708669,  1.66223253],
       [-0.24508304,  0.27864014]])

In [5]:
# 수치가 아니므로 수치작업 후 카테고리화
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(x[:, 0]) # 0번 나라 컬럼 fit
new_x = le.transform(x[:, 0])
new_x

x[:, 0] = new_x
x

array([[0, 44.0, 72000.0],
       [2, 27.0, 48000.0],
       [1, 30.0, 54000.0],
       [2, 38.0, 61000.0],
       [1, 40.0, nan],
       [0, 35.0, 58000.0],
       [2, nan, 52000.0],
       [0, 48.0, 79000.0],
       [1, 50.0, 83000.0],
       [0, 37.0, 67000.0]], dtype=object)

In [10]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categorical_features=[0])
# 카테고리: 지금 numpy데이터 몇번째 컬럼 바꿀건지 세팅을 미리해줘야 한다.
# x = ohe.fit_transform(x).toarray() # x를 통째로 넘기고 넘파이어레이로 한다.


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder, OneHotEncoder


path = 'dbs/DataPreprocess.csv'
df1 = pd.read_csv(path)
df1

x = df1.values[: , : -1]
y = df1.values[:, -1]

# 미싱 데이터 처리 
imputer = Imputer(missing_values='NaN', strategy="mean", axis=0) # NaN 데이터를 세로축 산술평균 
imputer.fit(x[:,1:3]) # 문자열 평균 못하니 제외 fit는 훈련만 실제적용은 transform
imputer.transform(x[:,1:3])
x[:,1:3] = imputer.transform(x[:,1:3])

# 피처 스케일링
sc_x = StandardScaler()
sc_x.fit(x[:,1:3])
x[:,1:3] = sc_x.transform(x[:,1:3])

# 원 핫 인코딩
le = LabelEncoder()
le.fit(x[:, 0]) # 0번 나라 컬럼 fit
x[:, 0] = le.transform(x[:, 0])

ohe = OneHotEncoder(categorical_features=[0])
# 카테고리: 지금 numpy데이터 몇번째 컬럼 바꿀건지 세팅을 미리해줘야 한다.
x = ohe.fit_transform(x).toarray() # x를 통째로 넘기고 넘파이어레이로 한다.

pd.DataFrame(x, columns=['France','Germany','Spain','Age','Salary'])



/Users/newdecide/.pyenv/versions/3.7.2/envs/3vm/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/newdecide/.pyenv/versions/3.7.2/envs/3vm/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/newdecide/.pyenv/versions/3.7.2/envs/3vm/lib/pytho

,France,Germany,Spain,Age,Salary
0,1.0,0.0,0.0,0.758874,7.494733e-01
1,0.0,0.0,1.0,-1.711504,-1.438178e+00
2,0.0,1.0,0.0,-1.275555,-8.912655e-01
3,0.0,0.0,1.0,-0.113024,-2.532004e-01
4,0.0,1.0,0.0,0.177609,6.632192e-16
5,1.0,0.0,0.0,-0.548973,-5.266569e-01
6,0.0,0.0,1.0,0.000000,-1.073570e+00
7,1.0,0.0,0.0,1.340140,1.387538e+00
8,0.0,1.0,0.0,1.630773,1.752147e+00
9,1.0,0.0,0.0,-0.258340,2.937125e-01
